In [511]:
import pandas as pd
import pyodbc
import sqlite3

# Connection

In [512]:
DB = {"servername": "MSI\\SQLEXPRESS",
      "database": "DEDSDatabase1"}

export_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER=' + DB['servername'] + 
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

In [513]:
def select_from(name, connection):
      dataframe = pd.read_sql_query("SELECT * FROM " + name, connection)
      
      columni = {}
      for column in dataframe.columns:
            if "TRIAL" in column:
                  dataframe.drop(column, axis=1, inplace=True)
            
            columni[column] = name.upper() + "_" + column.lower()
      dataframe.rename(columns=columni, inplace=True)
      return dataframe

def find_mutual_columns(dataset1, dataset2):
      column_names = set([])
      mutual_column_names = []
      for column in dataset1:
            column_names.add(column)
      for column in dataset2:
            if column in column_names:
                  mutual_column_names.append(column)
      return mutual_column_names

def merge_differing_columns_simple(dataframe1, dataframe2, key):
      # Pak de unieke columns uit de tweede dataframe
      column_names1 = set(dataframe1.columns)
      column_names2 = set(dataframe2.columns)
      uniques_from_2 = [key]
      for column in column_names2:
            if column not in column_names1:
                  uniques_from_2.append(column)
      
      # Merge deze met de eerste dataframe
      return pd.merge(dataframe1, dataframe2.loc[:, uniques_from_2], on=key, how='left')

def merge_differing_columns(dataframe1, dataframe2, left_key, right_key):
      # Pak de unieke columns uit de tweede dataframe
      column_names1 = set(dataframe1.columns)
      column_names2 = set(dataframe2.columns)
      uniques_from_2 = [right_key]
      for column in column_names2:
            if column not in column_names1:
                  uniques_from_2.append(column)
      
      # Merge deze met de eerste dataframe
      return pd.merge(dataframe1, dataframe2.loc[:, uniques_from_2], left_on=left_key, right_on=right_key, how='left')

def send_to_ssms(dataframe, target_name, cursor):
      column_string = "("
      i = 0
      for column in dataframe.columns:
            column_string += column
            if i == dataframe.columns.size - 1:
                  column_string += ")"
            else:
                  column_string += ", "
            i += 1
          
      
      for index, row in dataframe.iterrows():
            i = 0
            value_string = "("
            for column in dataframe.columns:
                  value_string += row[column]
                  if i == dataframe.columns.size - 1:
                        value_string += ")"
                  else:
                        value_string += ", "
                  i += 1
            query = "INSERT INTO " + target_name + " " + column_string + " VALUES " + value_string + ";"
            print(query)
            break
      #print(value_string)

In [514]:
connectie_sales = sqlite3.connect('go_sales.sqlite')
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"

product = select_from("product", connectie_sales)
product_type = select_from("product_type", connectie_sales)
product_line = select_from("product_line", connectie_sales)
SALES_sales_staff = select_from("sales_staff", connectie_sales)
SALES_sales_branch = select_from("sales_branch", connectie_sales)
SALES_retailer_site = select_from("retailer_site", connectie_sales)
SALES_country = select_from("country", connectie_sales)
order_header = select_from("order_header", connectie_sales)
order_details = select_from("order_details", connectie_sales)
order_method = select_from("order_method", connectie_sales)
target = select_from("SALES_TARGETData", connectie_sales)
returned_item = select_from("returned_item", connectie_sales)
return_reason = select_from("return_reason", connectie_sales)
sales_target = select_from("SALES_TARGETData", connectie_sales)

In [515]:
connectie_staff = sqlite3.connect('go_staff.sqlite')

course = select_from("course", connectie_staff)
STAFF_sales_staff = select_from("sales_staff", connectie_staff)
STAFF_sales_branch = select_from("sales_branch", connectie_staff)
satisfaction = select_from("satisfaction", connectie_staff)
satisfaction_type = select_from("satisfaction_type", connectie_staff)
training = select_from("training", connectie_staff)

In [516]:
connectie_crm = sqlite3.connect('go_crm.sqlite')

age_group = select_from("age_group", connectie_crm)
CRM_country = select_from("country", connectie_crm)
retailer = select_from("retailer", connectie_crm)
retailer_contact = select_from("retailer_contact", connectie_crm)
retailer_headquarters = select_from("retailer_headquarters", connectie_crm)
retailer_segment = select_from("retailer_segment", connectie_crm)
CRM_retailer_site = select_from("retailer_site", connectie_crm)
retailer_type = select_from("retailer_type", connectie_crm)
sales_demographic = select_from("sales_demographic", connectie_crm)
sales_territory = select_from("sales_territory", connectie_crm)

In [517]:
# CSV files
inventory_levels = pd.read_csv('GO_SALES_INVENTORY_LEVELSData.csv')
product_forecast = pd.read_csv('GO_SALES_PRODUCT_FORECASTData.csv')

In [518]:
# Merge similar tables
sales_staff = merge_differing_columns_simple(STAFF_sales_staff, SALES_sales_staff, 'SALES_STAFF_sales_staff_code')
sales_branch = merge_differing_columns_simple(SALES_sales_branch, STAFF_sales_branch, 'SALES_BRANCH_sales_branch_code')
country = merge_differing_columns_simple(SALES_country, CRM_country, 'COUNTRY_country_code')
retailer_site = merge_differing_columns_simple(SALES_retailer_site, CRM_retailer_site, 'RETAILER_SITE_retailer_site_code')

# Nina

In [519]:
# PRODUCT
output_PRODUCT = pd.merge(product, product_type, left_on='PRODUCT_product_type_code', right_on='PRODUCT_TYPE_product_type_code', how='left')
output_PRODUCT.drop('PRODUCT_TYPE_product_type_code', axis=1, inplace=True)
output_PRODUCT = pd.merge(output_PRODUCT, product_line, left_on='PRODUCT_TYPE_product_line_code', right_on='PRODUCT_LINE_product_line_code', how='left')
output_PRODUCT.drop('PRODUCT_LINE_product_line_code', axis=1, inplace=True)

# RETURN_REASON
output_RETURN_REASON = return_reason

# SALES_TARGETData
output_SALES_TARGETData = sales_target

# PRODUCT_FORECASTData
output_PRODUCT_FORECASTData = product_forecast

# INVENTORY_LEVELSData
output_INVENTORY_LEVELSData = inventory_levels

# SALES_STAFF
output_SALES_STAFF = pd.merge(sales_staff, sales_branch, left_on='SALES_STAFF_sales_branch_code', right_on='SALES_BRANCH_sales_branch_code')
output_SALES_STAFF.drop('SALES_BRANCH_sales_branch_code', axis=1)
output_SALES_STAFF = pd.merge(output_SALES_STAFF, country, left_on='SALES_BRANCH_country_code', right_on='COUNTRY_country_code')
output_SALES_STAFF.drop('COUNTRY_country_code', axis=1, inplace=True)

# ORDER_DETAILS
output_ORDER_DETAILS = pd.merge(order_details, order_header, left_on='ORDER_DETAILS_order_number', right_on='ORDER_HEADER_order_number')
output_ORDER_DETAILS.drop('ORDER_HEADER_order_number', axis=1, inplace=True)

# RETURNED_ITEM
output_RETURNED_ITEM = returned_item

# ORDER METHOD
output_ORDER_METHOD = order_method

send_to_ssms(returned_item, 'ORDER_DETAILS', export_cursor)

INSERT INTO ORDER_DETAILS (RETURNED_ITEM_return_code, RETURNED_ITEM_return_date, RETURNED_ITEM_order_detail_code, RETURNED_ITEM_return_reason_code, RETURNED_ITEM_return_quantity) VALUES (1491, 1-8-2021 04:10:24, 84858, 5, 8);


# Yemeserach

In [519]:
# RETAILER_SITE
output_RETAILER_SITE = pd.merge(retailer, retailer_type, left_on='RETAILER_retailer_type_code', right_on='RETAILER_TYPE_retailer_type_code', how='left')
output_RETAILER_SITE.drop('RETAILER_TYPE_retailer_type_code', axis=1, inplace=True)
output_RETAILER_SITE = pd.merge(output_RETAILER_SITE, retailer_site, left_on='RETAILER_retailer_code', right_on='RETAILER_SITE_retailer_code',  how='left')
output_RETAILER_SITE.drop('RETAILER_retailer_code', axis=1, inplace=True)

# COURSE
output_COURSE = course

# SATISFACTION_TYPE
output_SATISFACTION_TYPE = satisfaction_type

# SATISFACTION
output_SATISFACTION = satisfaction

# TRAINING 
output_training = training

# ORDER_METHOD 
output_order_method = order_method